In [ ]:
!apt-get -qq update
!apt-get -qq install -y ffmpeg
!pip -q install vosk datasets evaluate jiwer


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 96.9 MB/s eta 0:00:00


In [ ]:
!wget -q https://alphacephei.com/vosk/models/vosk-model-small-tr-0.3.zip
!unzip -q vosk-model-small-tr-0.3.zip

MODEL_PATH = "vosk-model-small-tr-0.3"


In [ ]:
from datasets import load_dataset, Audio

ds = load_dataset("sukumbasar/ASR_EchoBase_Raw", split="train")
ds = ds.cast_column("audio", Audio(decode=False))
print(ds)
print(ds[0]["audio"].keys())  # 'bytes' olmalı


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/461 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/10.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'text', 'speaker_id', 'is_common', 'sentence_index'],
    num_rows: 50
})
dict_keys(['bytes', 'path'])


In [ ]:
import os, tempfile, subprocess

def bytes_m4a_to_wav16k(bytes_data):
    # input m4a
    fin = tempfile.NamedTemporaryFile(suffix=".m4a", delete=False)
    fin.write(bytes_data)
    fin.flush()
    fin.close()

    # output wav
    fout = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    out_path = fout.name
    fout.close()

    cmd = ["ffmpeg","-y","-hide_banner","-loglevel","error",
           "-i", fin.name, "-ac","1", "-ar","16000", out_path]
    subprocess.run(cmd, check=True)

    # input temp temizle
    os.remove(fin.name)
    return out_path


In [ ]:
import json, wave
from vosk import Model, KaldiRecognizer

vosk_model = Model(MODEL_PATH)

def vosk_transcribe_wav(wav_path):
    wf = wave.open(wav_path, "rb")
    rec = KaldiRecognizer(vosk_model, wf.getframerate())
    rec.SetWords(True)

    text_parts = []
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            res = json.loads(rec.Result())
            text_parts.append(res.get("text",""))
    res = json.loads(rec.FinalResult())
    text_parts.append(res.get("text",""))
    wf.close()

    return " ".join([t for t in text_parts if t]).strip()


In [ ]:
import re, evaluate

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def normalize_tr_strict(t: str) -> str:
    t = t.lower()
    t = re.sub(r"[^a-zçğıöşü0-9\s]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t


In [ ]:
import os

preds, refs = [], []
bad = 0

for ex in ds:
    b = ex["audio"].get("bytes", None)
    if b is None:
        bad += 1
        continue

    try:
        wav_path = bytes_m4a_to_wav16k(b)
        pred = vosk_transcribe_wav(wav_path)
        os.remove(wav_path)

        preds.append(normalize_tr_strict(pred))
        refs.append(normalize_tr_strict(ex["text"]))
    except Exception as e:
        bad += 1

print("done:", len(preds), "bad:", bad)
print("WER:", wer_metric.compute(predictions=preds, references=refs))
print("CER:", cer_metric.compute(predictions=preds, references=refs))

# örnek çıktı
for i in range(min(5, len(preds))):
    print("="*60)
    print("REF :", refs[i])
    print("PRED:", preds[i])


done: 50 bad: 0
WER: 0.1349206349206349
CER: 0.027140672782874617
REF : bugün hava oldukça sakin
PRED: bugün hava oldukça sakin
REF : toplantı saatini üç buçuğa erteledim
PRED: toplantı saatini üç buçuğa erteledim
REF : yoğun trafik nedeniyle otobüs yarım saatten fazla gecikti
PRED: yoğun trafik nedeniyle otobüs yarım saatten fazla gecikti
REF : bu dosyayı ne zaman teslim etmemiz gerekiyor
PRED: bu dosyayı ne zaman teslim etmemiz gerekiyor
REF : e posta adresimi yanlış yazmış olabilirim tekrar kontrol eder misin
PRED: e posta adresimi yanlış yazmış olabilir tekrar kontrol eder misin
